In [14]:
# Import Module
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
from scipy import signal

In [15]:
# Read data, which has a size of N * 784 and N * 1
MNIST = h5py.File("..\MNISTdata.hdf5",'r')
x_train = np.float32(MNIST['x_train'][:])
x_test = np.float32(MNIST['x_test'][:])
y_train = np.int32(MNIST['y_train'][:,0])
y_test = np.int32(MNIST['y_test'][:,0])

In [16]:
# Reshape samples as 28 * 28 images
x_trainnew = np.reshape(x_train, (len(x_train),28,28))
x_testnew = np.reshape(x_test, (len(x_test),28,28))

In [17]:
# Build activate functions
relu = lambda x: x*(x>0)

# Input a m * n matrix, output a m * n matrix whose rows are transformed and normalized
def softmax(X):
    Xexp = np.exp(X)
    return Xexp / np.sum(Xexp,axis=1,keepdims=True)

In [18]:
# Initialize the parameters
def param_init(input_size, kernel_size, output_size):
    lx = input_size # 2-dim
    lk = kernel_size # 2-dim
    lh = (lx[0]-lk[0]+1, lx[1]-lk[1]+1) # Hidden layer size, 2-dim
    ly = output_size # 1-dim
    K = np.random.randn(lk[0],lk[1]) / max(lx)
    W = np.random.randn(ly,lh[0],lh[1]) / max(lx)
    b = np.zeros(ly)
    
    return K,W,b

In [36]:
# Build the forward step
# Model: Z = X * K → H = relu(Z) → U = WH + b → Yhat = softmax(U)
def Convolution(image, kernel): # Suppose images are given as (N,dim_1,dim_2)
    flag = 0 # Signify whether image has only 2 dims
    if len(image.shape)==2: # If only given (dim_1,dim_2), expand it to (1,dim_1,dim_2)
        flag = 1
        image = np.expand_dims(image,axis=0)
    N = image.shape[0]
    d1,d2 = image.shape[1:]
    k1,k2 = kernel.shape
    output_a = d1 - k1 + 1
    output_b = d2 - k2 + 1
    conv = np.zeros((N,output_a,output_b))
    for n in range(N):
        conv[n] = signal.correlate2d(image[n], kernel, mode="valid")
    if flag:
        conv = np.squeeze(conv)
    return conv # Output tensor shape (N,convdim_1,convdim_2)

def forward_prop(X,K,W,b):
    # Input to Hidden layer
    Z = Convolution(X,K) # Shape: (N, lx[0]-lk[0]+1, lx[1]-lk[1]+1)
    H = relu(Z) # Shape: (N, lx[0]-lk[0]+1, lx[1]-lk[1]+1)
    
    # Hidden layer to Output
    U = np.array([np.sum(np.multiply(W,Hobs), axis=(1,2)) for Hobs in H]) + b
    U.shape = (X.shape[0],W.shape[0]) # Shape: (N * ly)
    Yhat = softmax(U) # Shape: (N * ly)
    
    return Z, H, Yhat

In [20]:
# Build the back-propagation step
def back_prop(K,W,b,Z,H,Yhat,X,Y,alpha):
    batch_size = X.shape[0]
    UDel = Y - Yhat # Shape (N, ly)
    bDel = np.mean(UDel,axis=0,keepdims=True) # Shape (1, ly)
    WDel = np.tensordot(UDel.T, H, axes=1) / batch_size # Shape (ly, lx[0]-lk[0]+1, lx[1]-lk[1]+1)
    HDel = np.mean(np.tensordot(UDel, W, axes=1), axis=0, keepdims=True) # Shape (1, lx[0]-lk[0]+1, lx[1]-lk[1]+1)
    ZDel = np.multiply(HDel,(lambda x:(x>0))(Z)) # Shape (N, lx[0]-lk[0]+1, lx[1]-lk[1]+1)
    meanZDel = np.mean(ZDel, axis=0) # Shape (lx[0]-lk[0]+1, lx[1]-lk[1]+1)
    KDel = np.mean(np.array([Convolution(Xobs,meanZDel) for Xobs in X]),axis=0) # Shape: (lk[0], lk[1]) -- Mean of all samples
    
    bn = b + alpha * bDel # Length ly
    Wn = W + alpha * WDel # Shape (ly, lx[0]-lk[0]+1, lx[1]-lk[1]+1)
    Kn = K + alpha * KDel # Shape (1k[0], lk[1])
    
    return Kn,Wn,bn

In [35]:
# Build the complete Neural Network
def TwoLayer_CNN_train(X, Y, BatchSize = 1, ChannelSize = (3,3), num_channel = 1, OrigAlpha = 0.01, num_epochs = 10):    
    # Recode Y as One-Hot
    Y_oh = np.array(pd.get_dummies(np.squeeze(Y)))
    
    # Indicate number of units per layer
    N = X.shape[0] # Number of samples
    xsize = X.shape[1:] # Size of every sample
    ksize = ChannelSize # Size of the channel
    ysize = Y_oh.shape[1] # Number of classes
    
    # Initialized the parameters
    K,W,b = param_init(xsize,ksize,ysize)
    
    # Run 20 train iterations, record the error every time
    for epoch in range(num_epochs):
        if epoch <= 5:
            alpha = OrigAlpha
        elif epoch <= 10: 
            alpha = OrigAlpha * 1e-1
        elif epoch <= 15:
            alpha = OrigAlpha * 1e-2
        else:
            alpha = OrigAlpha * 1e-3
        total_cor = 0
        for n in range(int(N/6)):
            r = np.random.choice(N, size=BatchSize, replace=False)
            x_samp = X[[r]]
            y_samp = Y_oh[[r]]
            # Forward
            Z, H, Yhat = forward_prop(x_samp,K,W,b)
            pred = np.argmax(Yhat)
            if pred==Y[r]:
                total_cor += 1
            # Backward
            K,W,b = back_prop(K,W,b,Z,H,Yhat,x_samp,y_samp,alpha)
        print("Training Accuracy: ",total_cor / np.float(N/6))
    return K,W,b

In [37]:
K,W,b = TwoLayer_CNN_train(x_trainnew, y_train, OrigAlpha=0.01, num_epochs=5)

Training Accuracy:  0.8506
Training Accuracy:  0.893
Training Accuracy:  0.9005
Training Accuracy:  0.905
Training Accuracy:  0.9097


In [50]:
# For a given neural network, predict an input X
def predict_NN(X,K,W,b):
    X_predprob = forward_prop(X,K,W,b)[2]
    X_pred = X_predprob.argmax(axis=1) # Take the biggest probability as its choice
    return X_pred

In [53]:
# Predict on train set
y_predtrain = predict_NN(x_trainnew,K,W,b)
np.sum(y_predtrain == y_train) / x_trainnew.shape[0]

0.9117666666666666

In [54]:
# Predict on test set
# Still has problems!
y_predtest = predict_NN(x_testnew,K,W,b)
np.sum(y_predtest == y_test) / x_testnew.shape[0]

0.9117

In [7]:
Ut = np.array([1,2,3])
Ut.shape = (1,3)
Wt = np.array([[[1,1],[2,2]],[[3,3],[4,4]],[[5,5],[6,6]]])
Ht = np.array([[[0.1,0.1],[0.2,0.2]],[[0.3,0.3],[0.4,0.4]]])
kt = np.array([np.sum(np.multiply(Wt,H),axis=(1,2)) for H in Ht])
#np.tensordot(Ut,Wt,axes=1)

In [45]:
#r = np.random.choice(10, size=3, replace=False)
r = np.random.randint(10)
t = np.random.randn(10,5,5)
t[[r]].shape

(1, 5, 5)